In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet/

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/vehicle-detection.cfg
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/vehicle-detection.data
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/vehicle-detection.weights
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/vehicle-detection.names
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-detection-layout-classification.cfg
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-detection-layout-classification.data
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-detection-layout-classification.weights
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-detection-layout-classification.names
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-recognition.cfg
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-recognition.data
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-recognition.weights
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/lp-recognition.names
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/sample-image.jpg
!wget http://www.inf.ufpr.br/vri/databases/layout-independent-alpr/data/README.txt

In [ ]:
##usando como referencia https://stackoverflow.com/questions/64096953/how-to-convert-yolo-format-bounding-box-coordinates-into-opencv-format
import cv2
def crop_img(filename,Dict):
  box=Dict['relative_coordinates']
  img = cv2.imread(filename)
  dh, dw, _ = img.shape
  x=float(box['center_x'])
  w=float(box['width'] )
  y=float(box['center_y'])
  h=float(box['height'])
  x_min = int((x - w / 2) * dw)
  x_max = int((x + w / 2) * dw)
  y_min = int((y - h / 2) * dh)
  y_max = int((y + h / 2) * dh)
  if x_min>-1 and x_max>-1 and y_min>-1 and y_max>-1:
    crop_image = img[y_min:y_max, x_min:x_max]
    return crop_image

In [ ]:
import cv2
import json
def SelecionaPlaca(data,filename):
  for Dict  in data["objects"]:
    if Dict["confidence"]>0.5:
      crop_image = crop_img(data["filename"],Dict)
      cv2.imwrite("temporarioPlaca.jpg",crop_image)
      !./darknet detector test lp-recognition.data lp-recognition.cfg lp-recognition.weights -thresh .5 -ext_output  -dont_show -out temporarioValores.json <<< temporarioPlaca.jpg
      placa = cv2.imread('predictions.jpg')
      cv2.imwrite("placa"+filename,placa)
   ##   download("placa"+filename)
      imShow("placa"+filename)

In [ ]:
import cv2
import json
def identificaVeiculos(data,nome):
  for Dict  in data["objects"]:
    if Dict["confidence"]>0.5:
      crop_image = crop_img(data["filename"],Dict)
      if crop_image is None:
        print("esta falhou")
      else:
        newFilename=Dict["name"]+nome+data["filename"]
        cv2.imwrite(newFilename,crop_image)
        cv2.imwrite("temporarioVeiculo.jpg",crop_image)
        !./darknet detector test lp-detection-layout-classification.data lp-detection-layout-classification.cfg lp-detection-layout-classification.weights -thresh .25  -ext_output  -dont_show -out temporarioVeiculo.json <<< temporarioVeiculo.jpg
        veiculo = cv2.imread("predictions.jpg")
        print(newFilename)
        cv2.imwrite("predicao"+newFilename,veiculo)
  ##    download("predicao"+newFilename)
        imShow(newFilename)
        with open("temporarioVeiculo.json") as dados:
          dadosPlaca=json.load(dados)[0]
          SelecionaPlaca(dadosPlaca,newFilename)

In [ ]:
import json
def realizaPredicao(nome):
  img = cv2.imread(nome)
  cv2.imwrite("sample-image.jpg",img)
  !./darknet detector test vehicle-detection.data vehicle-detection.cfg vehicle-detection.weights -thresh .25 -ext_output -dont_show -out result.json   <<<sample-image.jpg
  imShow('predictions.jpg')
  imgPredita = cv2.imread('predictions.jpg')
  cv2.imwrite("predictions"+nome,imgPredita)
 ## download("predictions"+nome)
  with open("result.json") as dados:
    resultado=json.load(dados)
  print(resultado)
  identificaVeiculos(resultado[0],nome)

In [ ]:
realizaPredicao("sample-image.jpg")

In [ ]:
for x in range(1,113):
  realizaPredicao("imagem ("+str(x)+").jpg")

In [ ]:
!./darknet detector demo vehicle-detection.data vehicle-detection.cfg vehicle-detection.weights -thresh .25 -dont_show Video2min.mp4 -i 0 -out_filename Video2minResults.avi  -ext_output  -out result.json

In [ ]:
!./darknet detector demo vehicle-detection.data vehicle-detection.cfg vehicle-detection.weights -thresh .25 -dont_show Video2Minv2.mp4 -i 0 -ext_output -out result.json -out_filename Video2Minv2Results.avi